In [1]:
import requests
import pymongo

### Info on dataset and source
[link to source on energydata.info website!](https://energydata.info/dataset/mena-energy-indicators-2017)

"Various indicators for MENA countries to get an overview of the countries energy and economic profiles. The indicators are organized in 6 categories: economic indicators, energy indicators, oil indicators, gas indicators, electricity indicators & energy efficiency indicators. Each indicator includes name, unit, year, and value for that year. Main sources are the World Bank Group, the IMF, KNOEMA aggregating platform and EIA."

In [2]:
mena_resp = requests.get('https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/145369/pa-retp-indicators.json')
mena_json = mena_resp.json()
mena_resp

<Response [200]>

In [3]:
# list all countries found in this json list
[mena_json['countries'][country]['name'] for country in range(len(mena_json['countries']))]

['algeria',
 'bahrain',
 'egypt',
 'iraq',
 'jordan',
 'kuwait',
 'lebanon',
 'libya',
 'morocco',
 'oman',
 'qatar',
 'saudi arabia',
 'syria',
 'tunisia',
 'uae',
 'west bank and gaza',
 'yemen']

In [4]:
# show all data categories for
[item['type'] for item in mena_json['countries'][0]['subject']]

['economic indicators',
 'energy indicators',
 'oil indicators',
 'gas indicators',
 'electricity indicators',
 'energy efficiency indicators']

In [5]:
# item 0: economic indicators
    # 3 is gdp per capita
    # 5 is pop
    # 7 is HDI
    # 13 is co2 em itensity
    # 14 is co2 per capita
econ_indics = [3, 5, 7, 13, 14]
[mena_json['countries'][0]['subject'][0]['indicator'][sec] for sec in econ_indics]

[{'name': 'GDP per capita',
  'unit': 'US $',
  'values': [{'year': 2010, 'value': 4481},
   {'year': 2011, 'value': 5431},
   {'year': 2012, 'value': 5574},
   {'year': 2013, 'value': 5476},
   {'year': 2014, 'value': 5459},
   {'year': 2015, 'value': 4175},
   {'year': 2016, 'value': 4129},
   {'year': 2017, 'value': ''}]},
 {'name': 'Population',
  'unit': 'million',
  'values': [{'year': 2010, 'value': 36.04},
   {'year': 2011, 'value': 36.72},
   {'year': 2012, 'value': 37.44},
   {'year': 2013, 'value': 38.19},
   {'year': 2014, 'value': 38.93},
   {'year': 2015, 'value': 39.67},
   {'year': 2016, 'value': 40.38},
   {'year': 2017, 'value': ''}]},
 {'name': 'Human Development Index',
  'unit': '',
  'values': [{'year': 2010, 'value': 0.71},
   {'year': 2011, 'value': 0.72},
   {'year': 2012, 'value': 0.72},
   {'year': 2013, 'value': 0.72},
   {'year': 2014, 'value': 0.74},
   {'year': 2015, 'value': ''},
   {'year': 2016, 'value': ''},
   {'year': 2017, 'value': ''}]},
 {'name':

In [6]:
# item 1: energy indicators
    # 1 is gross energy production
    # 2-4 are fossil fuel prod
    # 5 is nuclear
    # 6 is gross eprod renewables
    # 7-10 are solar, wind, hydro, other energy prod
[mena_json['countries'][0]['subject'][1]['indicator'][sec] for sec in range(10)]

[{'name': 'Gross energy production',
  'unit': 'ktoe',
  'values': [{'year': 2010, 'value': 150510.895},
   {'year': 2011, 'value': 145832.565},
   {'year': 2012, 'value': 143763.68},
   {'year': 2013, 'value': 137669.492},
   {'year': 2014, 'value': 143197.3},
   {'year': 2015, 'value': ''},
   {'year': 2016, 'value': ''},
   {'year': 2017, 'value': ''}]},
 {'name': 'Gross energy production - Crude Oil and Oil Products',
  'unit': '%',
  'values': [{'year': 2010, 'value': 52.15272223316458},
   {'year': 2011, 'value': 52.25005539743472},
   {'year': 2012, 'value': 49.515179355453334},
   {'year': 2013, 'value': 49.92558990484253},
   {'year': 2014, 'value': 50.96184564932439},
   {'year': 2015, 'value': ''},
   {'year': 2016, 'value': ''},
   {'year': 2017, 'value': ''}]},
 {'name': 'Gross energy production - Natural Gas',
  'unit': '%',
  'values': [{'year': 2010, 'value': 47.80292217383998},
   {'year': 2011, 'value': 47.70930415987677},
   {'year': 2012, 'value': 50.43678208571178}

In [7]:
# item 4: electricity indicators
    # 8 is kWh consumption per capita
    # 9 is cost of elec per kWh
[mena_json['countries'][0]['subject'][4]['indicator'][sec] for sec in (8, 9)]

[{'name': 'Cost of electricity',
  'unit': '$/kWh',
  'values': [{'year': 2010, 'value': ''},
   {'year': 2011, 'value': ''},
   {'year': 2012, 'value': 0.2016},
   {'year': 2013, 'value': 0.2157},
   {'year': 2014, 'value': 0.1984},
   {'year': 2015, 'value': 0.1563},
   {'year': 2016, 'value': 0.1184},
   {'year': 2017, 'value': ''}]},
 {'name': 'Electricity price',
  'unit': '$/kWh',
  'values': [{'year': 2010, 'value': ''},
   {'year': 2011, 'value': ''},
   {'year': 2012, 'value': 0.03868},
   {'year': 2013, 'value': 0.04012},
   {'year': 2014, 'value': 0.04014},
   {'year': 2015, 'value': 0.03229},
   {'year': 2016, 'value': 0.03964},
   {'year': 2017, 'value': ''}]}]

## Restructuring dataset to something mongo will accept
"Operation passed in cannot be an Array"

In [8]:
# connect to db for this assignment
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client.hw3
# create collection for country docs
countries = client.countries
# add each country doc to the collection
for country in mena_json['countries']:
    db.countries.insert_one(country)
    print(f'inserted {country["name"]} into collection.')

inserted algeria into collection.
inserted bahrain into collection.
inserted egypt into collection.
inserted iraq into collection.
inserted jordan into collection.
inserted kuwait into collection.
inserted lebanon into collection.
inserted libya into collection.
inserted morocco into collection.
inserted oman into collection.
inserted qatar into collection.
inserted saudi arabia into collection.
inserted syria into collection.
inserted tunisia into collection.
inserted uae into collection.
inserted west bank and gaza into collection.
inserted yemen into collection.


## Exploring the data: motivating questions
1. Which countries in the dataset have the greatest percentage of their total energy output as solar energy?

In [9]:
mena_json['countries'][0]['subject'][1]['indicator'][6]

{'name': 'Gross energy production - Solar',
 'unit': '%',
 'values': [{'year': 2010, 'value': 0},
  {'year': 2011, 'value': 0},
  {'year': 2012, 'value': 0},
  {'year': 2013, 'value': 0},
  {'year': 2014, 'value': 0},
  {'year': 2015, 'value': ''},
  {'year': 2016, 'value': ''},
  {'year': 2017, 'value': ''}]}

In [10]:
# 1. Which countries in the dataset have the most solar energy output?
# drill down to get solar amounts per country!
most_solar = db.countries.aggregate([
    {
        # expand elements of subject list for each country
        "$unwind": "$subject"
    },
    {
        # extract 'energy indicators' subject
        "$match": {
            "subject.type": "energy indicators"
        }
    },
    {
        # expand elements of indicator list for this subject type
        "$unwind": "$subject.indicator"
    },
    {
        # extract 'Gross energy production - Solar' indicator
        "$match": {
            "subject.indicator.name": "Gross energy production - Solar"
        }
    },
    {
        # expand elements of year list for this indicator
        "$unwind": "$subject.indicator.values"
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": ""}}
    },
    {
        # for each country, get the max value for annual solar energy production
        "$group": {
            "_id": "$name",
            "maxSolarProduction": {"$max": "$subject.indicator.values.value"}
        }
    },
    {
        "$sort": {
            "maxSolarProduction": -1
        }
    }
])

[country for country in most_solar]

[{'_id': 'jordan', 'maxSolarProduction': 58.33045692677052},
 {'_id': 'lebanon', 'maxSolarProduction': 15.153580203689925},
 {'_id': 'tunisia', 'maxSolarProduction': 0.6787589293911857},
 {'_id': 'uae', 'maxSolarProduction': 0.034140968315127765},
 {'_id': 'egypt', 'maxSolarProduction': 0.026434563845563263},
 {'_id': 'saudi arabia', 'maxSolarProduction': 1.3995593650078733e-05},
 {'_id': 'qatar', 'maxSolarProduction': 0},
 {'_id': 'bahrain', 'maxSolarProduction': 0},
 {'_id': 'morocco', 'maxSolarProduction': 0},
 {'_id': 'libya', 'maxSolarProduction': 0},
 {'_id': 'kuwait', 'maxSolarProduction': 0},
 {'_id': 'syria', 'maxSolarProduction': 0},
 {'_id': 'yemen', 'maxSolarProduction': 0},
 {'_id': 'oman', 'maxSolarProduction': 0},
 {'_id': 'iraq', 'maxSolarProduction': 0},
 {'_id': 'algeria', 'maxSolarProduction': 0}]


2. Which countries in the dataset have the lowest energy usage per capita?
- note: TOE is "tonne of oil equivalent" and the figures are per capita per year

In [11]:
mena_json['countries'][0]['subject'][1]['indicator'][30]

{'name': 'Energy use per capita',
 'unit': 'toe',
 'values': [{'year': 2010, 'value': 0.7358203107658158},
  {'year': 2011, 'value': 0.7528700163398693},
  {'year': 2012, 'value': 0.8144358974358974},
  {'year': 2013, 'value': 0.8351275202932705},
  {'year': 2014, 'value': 0.8995416388389418},
  {'year': 2015, 'value': ''},
  {'year': 2016, 'value': ''},
  {'year': 2017, 'value': ''}]}

In [12]:
least_usage_pc = db.countries.aggregate([
    {
        # expand elements of subject list for each country
        "$unwind": "$subject"
    },
    {
        # extract 'energy indicators' subject
        "$match": {
            "subject.type": "energy indicators"
        }
    },
    {
        # expand elements of indicator list for this subject type
        "$unwind": "$subject.indicator"
    },
    {
        # extract energy use per capita indicator
        "$match": {
            "subject.indicator.name": "Energy use per capita"
        }
    },
    {
        # expand elements of year list for this indicator
        "$unwind": "$subject.indicator.values"
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": ""}}
    },
    {
        # for each country, get the max value for annual solar energy production
        "$group": {
            "_id": "$name",
            "toePerCapita": {"$max": "$subject.indicator.values.value"}
        }
    },
    {
        "$sort": {
            "toePerCapita": 1
        }
    }
])

[country for country in least_usage_pc]

[{'_id': 'yemen', 'toePerCapita': 0.23276952604778692},
 {'_id': 'morocco', 'toePerCapita': 0.43197437841115827},
 {'_id': 'syria', 'toePerCapita': 0.6535419401544401},
 {'_id': 'egypt', 'toePerCapita': 0.660383410908223},
 {'_id': 'tunisia', 'toePerCapita': 0.70396},
 {'_id': 'jordan', 'toePerCapita': 0.7496350500715308},
 {'_id': 'iraq', 'toePerCapita': 0.7868724211165048},
 {'_id': 'lebanon', 'toePerCapita': 0.8898365853658536},
 {'_id': 'algeria', 'toePerCapita': 0.8995416388389418},
 {'_id': 'libya', 'toePerCapita': 2.1118398724082934},
 {'_id': 'bahrain', 'toePerCapita': 4.495690161527166},
 {'_id': 'saudi arabia', 'toePerCapita': 4.586784525736484},
 {'_id': 'oman', 'toePerCapita': 4.768878873239437},
 {'_id': 'kuwait', 'toePerCapita': 4.996120760233918},
 {'_id': 'uae', 'toePerCapita': 5.558555370985603},
 {'_id': 'qatar', 'toePerCapita': 8.554577419354839}]

3. Which countries have the most diverse range of energy generation solutions?

In [13]:
[indicator['name'] for indicator in mena_json['countries'][0]['subject'][1]['indicator']][:10]

['Gross energy production',
 'Gross energy production - Crude Oil and Oil Products',
 'Gross energy production - Natural Gas',
 'Gross energy production - Coal',
 'Gross energy production - Nuclear',
 'Gross energy production - Renewables',
 'Gross energy production - Solar',
 'Gross energy production - Wind',
 'Gross energy production - Hydropower',
 'Gross energy production - Other']

In [14]:
most_diverse_renewables = db.countries.aggregate([
    {
        # expand elements of subject list for each country
        "$unwind": "$subject"
    },
    {
        # extract 'energy indicators' subject
        "$match": {
            "subject.type": "energy indicators"
        }
    },
    {
        # expand elements of indicator list for this subject type
        "$unwind": "$subject.indicator"
    },
    {
        # extract 'Gross energy production - Solar' indicator
        "$match": {
            "subject.indicator.name": {"$regex": "^Gross energy production -*"}
        }
    },
    {
        # expand elements of year list for this indicator
        "$unwind": "$subject.indicator.values"
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": ""}}
        # and-ing didn't work: {"$and": [{"$ne": ""}, {"$ne": 0}]}
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": 0}}
    },
    {
        # rewind all non-zero, non-empty energy prod methods for each country
        "$group": {
            "_id": {
                "country": "$name",
                "energy production method": "$subject.indicator.name"
            },
            "energyGenerator": {"$first": "$subject.indicator.name"}
        }
    },

    {
        # for each country, list the unique energy production methods (using self-defined agg fields above)
        "$group": {
            "_id": "$_id.country",
            "energyGenerators": {"$push": "$energyGenerator"}
        }
    },
])

# TODO if i have extra time: get substrings to exclude common prefix, rank methods by output, and include numbers for each
[country for country in most_diverse_renewables]

[{'_id': 'tunisia',
  'energyGenerators': ['Gross energy production - Solar',
   'Gross energy production - Wind',
   'Gross energy production - Renewables',
   'Gross energy production - Natural Gas',
   'Gross energy production - Hydropower',
   'Gross energy production - Crude Oil and Oil Products',
   'Gross energy production - Other']},
 {'_id': 'bahrain',
  'energyGenerators': ['Gross energy production - Other',
   'Gross energy production - Crude Oil and Oil Products',
   'Gross energy production - Natural Gas']},
 {'_id': 'algeria',
  'energyGenerators': ['Gross energy production - Crude Oil and Oil Products',
   'Gross energy production - Natural Gas',
   'Gross energy production - Hydropower',
   'Gross energy production - Renewables',
   'Gross energy production - Other']},
 {'_id': 'qatar',
  'energyGenerators': ['Gross energy production - Natural Gas',
   'Gross energy production - Crude Oil and Oil Products',
   'Gross energy production - Other']},
 {'_id': 'iraq',
  'ene

4. Which countries have the lowest recorded average electricity cost? (note: lots of missing data, taking mean of any data present for each country's price per kWh)

In [15]:
cost_per_kwh = db.countries.aggregate([
    {
        # expand elements of subject list for each country
        "$unwind": "$subject"
    },
    {
        # extract 'energy indicators' subject
        "$match": {
            "subject.type": "electricity indicators"
        }
    },
    {
        # expand elements of indicator list for this subject type
        "$unwind": "$subject.indicator"
    },
    {
        # extract 'Gross energy production - Solar' indicator
        "$match": {
            "subject.indicator.name": "Cost of electricity"
        }
    },
    {
        # expand elements of year list for this indicator
        "$unwind": "$subject.indicator.values"
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": ""}}
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": 0}}
    },
    {
        # for each country, get the mean value for energy cost per kWh
        "$group": {
            "_id": "$name",
            "costPer_kWh": {"$avg": "$subject.indicator.values.value"}
        }
    },
    {
        "$sort": {
            "costPer_kWh": 1
        }
    }
])

[country for country in cost_per_kwh]

[{'_id': 'oman', 'costPer_kWh': 0.06578},
 {'_id': 'uae', 'costPer_kWh': 0.103},
 {'_id': 'qatar', 'costPer_kWh': 0.1151},
 {'_id': 'kuwait', 'costPer_kWh': 0.1295},
 {'_id': 'tunisia', 'costPer_kWh': 0.13778030842250014},
 {'_id': 'algeria', 'costPer_kWh': 0.17808},
 {'_id': 'saudi arabia', 'costPer_kWh': 0.21329999999999996}]

5. Which countries have not invested in solar at all? (assume that no data and zero are equivalent)

In [16]:
no_solar = db.countries.aggregate([
    {
        # expand elements of subject list for each country
        "$unwind": "$subject"
    },
    {
        # extract 'energy indicators' subject
        "$match": {
            "subject.type": "energy indicators"
        }
    },
    {
        # expand elements of indicator list for this subject type
        "$unwind": "$subject.indicator"
    },
    {
        # extract 'Gross energy production - Solar' indicator
        "$match": {
            "subject.indicator.name": "Gross energy production - Solar"
        }
    },
    {
        # expand elements of year list for this indicator
        "$unwind": "$subject.indicator.values"
    },
    {
        # exclude years with empty string as value
        "$match": {"$or":
            [
            {"subject.indicator.values.value": {"$eq": ""}},
            {"subject.indicator.values.value": {"$eq": 0}}
            ]
        }
    },

    {
        # for each country, get the max value for annual solar energy production
        "$group": {
            "_id": "$name",
            "no_solar": {"$max": "$subject.indicator.values.value"}
        }
    },
    {
        "$sort": {
            "no_solar": 1
        }
    }
])

[country for country in no_solar]

[{'_id': 'syria', 'no_solar': ''},
 {'_id': 'uae', 'no_solar': ''},
 {'_id': 'yemen', 'no_solar': ''},
 {'_id': 'oman', 'no_solar': ''},
 {'_id': 'iraq', 'no_solar': ''},
 {'_id': 'algeria', 'no_solar': ''},
 {'_id': 'bahrain', 'no_solar': ''},
 {'_id': 'qatar', 'no_solar': ''},
 {'_id': 'tunisia', 'no_solar': ''},
 {'_id': 'lebanon', 'no_solar': ''},
 {'_id': 'morocco', 'no_solar': ''},
 {'_id': 'libya', 'no_solar': ''},
 {'_id': 'west bank and gaza', 'no_solar': ''},
 {'_id': 'egypt', 'no_solar': ''},
 {'_id': 'jordan', 'no_solar': ''},
 {'_id': 'kuwait', 'no_solar': ''},
 {'_id': 'saudi arabia', 'no_solar': ''}]

6. which countries has experienced the greatest increase in renewables (measures as a percent increase)?

In [17]:
greatest_increase_solar = db.countries.aggregate([
    {
        # expand elements of subject list for each country
        "$unwind": "$subject"
    },
    {
        # extract 'energy indicators' subject
        "$match": {
            "subject.type": "energy indicators"
        }
    },
    {
        # expand elements of indicator list for this subject type
        "$unwind": "$subject.indicator"
    },
    {
        # extract 'Gross energy production - Solar' indicator
        "$match": {
            "subject.indicator.name": "Gross energy production - Renewables"
        }
    },
    {
        # expand elements of year list for this indicator
        "$unwind": "$subject.indicator.values"
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": ""}}
    },
    {
        "$group": {
            "_id": "$name",
            # agg yearly gross production of renewables estimates into min/max per country
            "min_gpr": {"$min": "$subject.indicator.values.value"},
            "max_gpr": {"$max": "$subject.indicator.values.value"}
        }
    },
    {
        "$addFields": {
            "diff_renewables": {
                "$subtract": ["$max_gpr", "$min_gpr"]
            },
        }
    },
    {
        "$sort": {
            "diff_renewables": -1
        }
    }
])

[country for country in greatest_increase_solar]

[{'_id': 'jordan',
  'min_gpr': 49.574014323103064,
  'max_gpr': 62.223577329314026,
  'diff_renewables': 12.649563006210961},
 {'_id': 'tunisia',
  'min_gpr': 13.215865058801754,
  'max_gpr': 17.421956137328102,
  'diff_renewables': 4.206091078526349},
 {'_id': 'syria',
  'min_gpr': 0.8285816430336242,
  'max_gpr': 4.699126103018552,
  'diff_renewables': 3.8705444599849272},
 {'_id': 'morocco',
  'min_gpr': 94.93433768735109,
  'max_gpr': 97.17498285626148,
  'diff_renewables': 2.2406451689103903},
 {'_id': 'egypt',
  'min_gpr': 3.3845212693472475,
  'max_gpr': 3.825001246003533,
  'diff_renewables': 0.44047997665628547},
 {'_id': 'libya',
  'min_gpr': 0.15077244777020282,
  'max_gpr': 0.4773023313106763,
  'diff_renewables': 0.32652988354047346},
 {'_id': 'yemen',
  'min_gpr': 0.5351124652651305,
  'max_gpr': 0.7275874812992533,
  'diff_renewables': 0.19247501603412287},
 {'_id': 'iraq',
  'min_gpr': 0.18184859499263575,
  'max_gpr': 0.360879145043363,
  'diff_renewables': 0.17903055

7. which countries has experienced the greatest decrease in fossil fuels (measures as a percent decrease)?

In [18]:
[indicator['name'] for indicator in mena_json['countries'][0]['subject'][1]['indicator']][:10]

['Gross energy production',
 'Gross energy production - Crude Oil and Oil Products',
 'Gross energy production - Natural Gas',
 'Gross energy production - Coal',
 'Gross energy production - Nuclear',
 'Gross energy production - Renewables',
 'Gross energy production - Solar',
 'Gross energy production - Wind',
 'Gross energy production - Hydropower',
 'Gross energy production - Other']

In [19]:
greatest_decrease_ff = db.countries.aggregate([
    {
        # expand elements of subject list for each country
        "$unwind": "$subject"
    },
    {
        # extract 'energy indicators' subject
        "$match": {
            "subject.type": "energy indicators"
        }
    },
    {
        # expand elements of indicator list for this subject type
        "$unwind": "$subject.indicator"
    },
    {
        # retain all (and only) the fossil fuel indicators
        "$match": {
            "$or": [
                {"subject.indicator.name": "Gross energy production - Crude Oil and Oil Products"},
                {"subject.indicator.name": "Gross energy production - Natural Gas"},
                {"subject.indicator.name": "Gross energy production - Coal"},
            ]
        }
    },
    {
        # expand elements of year list for this indicator
        "$unwind": "$subject.indicator.values"
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": ""}}
    },
    {
        "$group": {
            "_id": {
                "country": "$name",
                "energy_production_method": "$subject.indicator.name"
            },
            # agg yearly percentage values for each indicator's values
            "min_pct": {"$min": "$subject.indicator.values.value"},
            "max_pct": {"$max": "$subject.indicator.values.value"}
        }
    },
    {
        # group by country, sum mins and maxes of groups per country
        "$group": {
            "_id": "$_id.country",
            # sum mins and maxes for different fossil fuel types to get a ff agg
            "ff_agg_min": {"$sum": "$min_pct"},
            "ff_agg_max": {"$sum": "$max_pct"}
        }
    },
    {
        # add new field of fossil fuel agg diffs
        "$addFields": {
            "ff_decrease_pct": {
                "$subtract": ["$ff_agg_max", "$ff_agg_min"]
            },
        }
    },
    {
        "$sort": {
            "ff_decrease_pct": -1
        }
    }
])

# some output values are quite different from what I expected to be - likely an issue with summing min and max percentages
[country for country in greatest_decrease_ff]

[{'_id': 'syria',
  'ff_agg_min': 50.26239000017762,
  'ff_agg_max': 143.88757934908725,
  'ff_decrease_pct': 93.62518934890963},
 {'_id': 'yemen',
  'ff_agg_min': 75.05867210200753,
  'ff_agg_max': 123.69683239348643,
  'ff_decrease_pct': 48.6381602914789},
 {'_id': 'libya',
  'ff_agg_min': 83.0315467078108,
  'ff_agg_max': 116.37350445196225,
  'ff_decrease_pct': 33.34195774415144},
 {'_id': 'jordan',
  'ff_agg_min': 37.755932876214075,
  'ff_agg_max': 50.44276474157611,
  'ff_decrease_pct': 12.686831865362038},
 {'_id': 'qatar',
  'ff_agg_min': 95.29806585378168,
  'ff_agg_max': 104.70193414621832,
  'ff_decrease_pct': 9.403868292436641},
 {'_id': 'egypt',
  'ff_agg_min': 92.57821640973447,
  'ff_agg_max': 99.94372052468847,
  'ff_decrease_pct': 7.365504114954007},
 {'_id': 'tunisia',
  'ff_agg_min': 80.16216526356604,
  'ff_agg_max': 86.62491638961389,
  'ff_decrease_pct': 6.4627511260478485},
 {'_id': 'algeria',
  'ff_agg_min': 97.2244835153301,
  'ff_agg_max': 102.6868374831465,


8. Which year saw the greatest average decrease in cost per kWh in these countries?
9. Which countries have the greatest ratio of renewable energy production to crude oil production?
10. Which countries are producing the most fossil fuels per capita (annual ouput divided by population)?

## Datavis question
1. what's the relationship between co2 per capita and GDP per capita? add point for each country on a 2-axis plot

and is it equivalent to the provided emissions intensity?
economic indicators ->
'name': 'CO2 emissions intensity',
'unit': 'kg/$1000 of GDP',

In [20]:
# retrieving co2 per capita
co2_per_capita = db.countries.aggregate([
    {
        # expand elements of subject list for each country
        "$unwind": "$subject"
    },
    {
        # extract 'energy indicators' subject
        "$match": {
            "subject.type": "economic indicators"
        }
    },
    {
        # expand elements of indicator list for this subject type
        "$unwind": "$subject.indicator"
    },
    {
        # extract 'Gross energy production - Solar' indicator
        "$match": {
            "subject.indicator.name": "CO2 emissions per capita"
        }
    },
    {
        # expand elements of year list for this indicator
        "$unwind": "$subject.indicator.values"
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": ""}}
    },
    {
        # exclude years with zero as value to avoid sandbagging avg
        "$match": {"subject.indicator.values.value": {"$ne": 0}}
    },
    {
        # for each country, get the avg value for annual co2 emissions per capita
        "$group": {
            "_id": "$name",
            "avg_co2_pc_tons": {"$avg": "$subject.indicator.values.value"}
        }
    },
    {
        "$sort": {
            "avg_co2_pc_tons": -1
        }
    }
])

co2_pc_list = [country for country in co2_per_capita]
co2_pc_list

[{'_id': 'qatar', 'avg_co2_pc_tons': 40.236666666666665},
 {'_id': 'kuwait', 'avg_co2_pc_tons': 26.241666666666667},
 {'_id': 'uae', 'avg_co2_pc_tons': 20.91},
 {'_id': 'oman', 'avg_co2_pc_tons': 19.871666666666666},
 {'_id': 'saudi arabia', 'avg_co2_pc_tons': 15.436666666666667},
 {'_id': 'bahrain', 'avg_co2_pc_tons': 15.395000000000001},
 {'_id': 'libya', 'avg_co2_pc_tons': 8.323333333333332},
 {'_id': 'iraq', 'avg_co2_pc_tons': 4.283333333333333},
 {'_id': 'lebanon', 'avg_co2_pc_tons': 4.175000000000001},
 {'_id': 'algeria', 'avg_co2_pc_tons': 3.4483333333333333},
 {'_id': 'jordan', 'avg_co2_pc_tons': 3.0933333333333333},
 {'_id': 'syria', 'avg_co2_pc_tons': 2.586666666666667},
 {'_id': 'egypt', 'avg_co2_pc_tons': 2.5566666666666666},
 {'_id': 'tunisia', 'avg_co2_pc_tons': 2.393333333333333},
 {'_id': 'morocco', 'avg_co2_pc_tons': 1.6716666666666666},
 {'_id': 'yemen', 'avg_co2_pc_tons': 1.0766666666666667}]

In [21]:
# retrieving GDP per capita
gdp_per_capita = db.countries.aggregate([
    {
        # expand elements of subject list for each country
        "$unwind": "$subject"
    },
    {
        # extract 'energy indicators' subject
        "$match": {
            "subject.type": "economic indicators"
        }
    },
    {
        # expand elements of indicator list for this subject type
        "$unwind": "$subject.indicator"
    },
    {
        # extract 'Gross energy production - Solar' indicator
        "$match": {
            "subject.indicator.name": "GDP per capita"
        }
    },
    {
        # expand elements of year list for this indicator
        "$unwind": "$subject.indicator.values"
    },
    {
        # exclude years with empty string as value
        "$match": {"subject.indicator.values.value": {"$ne": ""}}
    },
    {
        # exclude years with zero as value to avoid sandbagging avg
        "$match": {"subject.indicator.values.value": {"$ne": 0}}
    },
    {
        # for each country, get the avg value for GDP per capita
        "$group": {
            "_id": "$name",
            "avg_gdp_per_capita": {"$avg": "$subject.indicator.values.value"}
        }
    },
    {
        "$sort": {
            "avg_gdp_per_capita": -1
        }
    }
])

gdp_pc_list = [country for country in gdp_per_capita]
gdp_pc_list

[{'_id': 'qatar', 'avg_gdp_per_capita': 85974.71428571429},
 {'_id': 'uae', 'avg_gdp_per_capita': 40105.0},
 {'_id': 'kuwait', 'avg_gdp_per_capita': 37058.142857142855},
 {'_id': 'bahrain', 'avg_gdp_per_capita': 24397.714285714286},
 {'_id': 'saudi arabia', 'avg_gdp_per_capita': 22523.714285714286},
 {'_id': 'oman', 'avg_gdp_per_capita': 20112.0},
 {'_id': 'lebanon', 'avg_gdp_per_capita': 10288.0},
 {'_id': 'libya', 'avg_gdp_per_capita': 8765.142857142857},
 {'_id': 'iraq', 'avg_gdp_per_capita': 5651.285714285715},
 {'_id': 'algeria', 'avg_gdp_per_capita': 4960.714285714285},
 {'_id': 'jordan', 'avg_gdp_per_capita': 4612.714285714285},
 {'_id': 'tunisia', 'avg_gdp_per_capita': 4120.142857142857},
 {'_id': 'egypt', 'avg_gdp_per_capita': 3317.3333333333335},
 {'_id': 'morocco', 'avg_gdp_per_capita': 3109.4285714285716},
 {'_id': 'syria', 'avg_gdp_per_capita': 2807.0},
 {'_id': 'west bank and gaza', 'avg_gdp_per_capita': 2768.3333333333335},
 {'_id': 'yemen', 'avg_gdp_per_capita': 1328.42

## I ran out of time to do the visualization but the data is here
## there's a clear positive correlation between co2 per capita and GDP per capita in these nations - depressing but real

In [22]:
import matplotlib.pyplot as plt

# co2_pc_list
[{country['_id']: country['avg_gdp_per_capita']} for country in gdp_pc_list]

[{'qatar': 85974.71428571429},
 {'uae': 40105.0},
 {'kuwait': 37058.142857142855},
 {'bahrain': 24397.714285714286},
 {'saudi arabia': 22523.714285714286},
 {'oman': 20112.0},
 {'lebanon': 10288.0},
 {'libya': 8765.142857142857},
 {'iraq': 5651.285714285715},
 {'algeria': 4960.714285714285},
 {'jordan': 4612.714285714285},
 {'tunisia': 4120.142857142857},
 {'egypt': 3317.3333333333335},
 {'morocco': 3109.4285714285716},
 {'syria': 2807.0},
 {'west bank and gaza': 2768.3333333333335},
 {'yemen': 1328.4285714285713}]